# Deployment finder
A script to list all the Azure OpenAI model deployments in an Azure subscription
***
<br><br>
Import libraries, and load the Azure Subscription ID

In [14]:
import os
from azure.identity import DefaultAzureCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.resource import SubscriptionClient
import pandas as pd
from itables import show
from dotenv import load_dotenv

load_dotenv()

# Get the Azure subscription ID from the .env file
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]

Authenticate with Azure

In [15]:
credential = DefaultAzureCredential()

Optionally, list all the available Azure Subscriptions IDs 

In [16]:
"""
# Create a SubscriptionClient
subscriptions_client = SubscriptionClient(credential)

# List all Azure subscriptions
subscriptions = subscriptions_client.subscriptions.list()

# Output subscription details
for subscription in subscriptions:
    display(f"Subscription ID: {subscription.subscription_id}, Subscription Name: {subscription.display_name}")
"""

'\n# Create a SubscriptionClient\nsubscriptions_client = SubscriptionClient(credential)\n\n# List all Azure subscriptions\nsubscriptions = subscriptions_client.subscriptions.list()\n\n# Output subscription details\nfor subscription in subscriptions:\n    display(f"Subscription ID: {subscription.subscription_id}, Subscription Name: {subscription.display_name}")\n'

Create the Resource Management and Cognitive Services Management clients

In [17]:
# Create a Cognitive Services Management client
client = CognitiveServicesManagementClient(credential, subscription_id)

# Initialize the Resource Management client
resource_client = ResourceManagementClient(credential, subscription_id)

Iterate through the resource groups and Azure OpenAI services to hydrate the dataframe

In [18]:
# Initialize an empty DataFrame with the specified columns
deployment_df = pd.DataFrame(columns=["Resource Group", "Azure OpenAI Service Name", "Deployment Name", "Model Name", "Version"])

# Filter for OpenAI instances and list model deployments
for resource_group in resource_client.resource_groups.list():
    accounts = client.accounts.list_by_resource_group(resource_group.name)  # Corrected line
    for account in accounts:
        if account.kind == 'OpenAI':
            # Correctly access deployment properties using dot notation
            deployments = client.deployments.list(resource_group.name, account.name)
            deployment_info = [{"deploymentname": deployment.name, "modelname": deployment.properties.model.name, "version": deployment.properties.model.version} for deployment in deployments]
            
            # output deployment_info in a more readable format
            for deployment in deployment_info:
                #print(f"Deployment Name: {deployment['deploymentname']}, Model Name: {deployment['modelname']}, Version: {deployment['version']}")
                # Inside the loop where you add rows to the DataFrame
                new_row = pd.DataFrame([{
                    "Resource Group": resource_group.name,
                    "Azure OpenAI Service Name": account.name,
                    "Deployment Name": deployment['deploymentname'],
                    "Model Name": deployment['modelname'],
                    "Version": deployment['version']
                }])
                
                deployment_df = pd.concat([deployment_df, new_row], ignore_index=True)

Display the dataframe as an interactive table

In [20]:
show(deployment_df, paging=False)